In [ ]:
from pathlib import Path
import sqlite3
import pandas as pd
import numpy as np

# ========= 1. Paths =========
price_db = Path("data") / "processed" / "data_processed.sqlite"
sma_db   = Path("data") / "signals" / "SMAsignal.sqlite"
ema_db   = Path("data") / "signals" / "EMAsignal.sqlite"

# ========= 2. Load Close prices =========
with sqlite3.connect(price_db) as conn:
    tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
    if tables.empty:
        raise RuntimeError(f"No tables found in {price_db}")
    table_name = tables["name"].iloc[0]
    price_df = pd.read_sql(
        f'SELECT Date, Close FROM "{table_name}"',
        conn,
        parse_dates=["Date"],
    )

price_df = price_df.set_index("Date").sort_index()
price_df = price_df[["Close"]]

# ========= 3. Load SMA signals =========
with sqlite3.connect(sma_db) as conn:
    sma_df = pd.read_sql("SELECT Date, signalSMA, positionSMA FROM signals_sma",
                         conn, parse_dates=["Date"])

sma_df = sma_df.set_index("Date").sort_index()

# ========= 4. Load EMA signals =========
with sqlite3.connect(ema_db) as conn:
    ema_df = pd.read_sql("SELECT Date, signalEMA, positionEMA FROM signals_ema",
                         conn, parse_dates=["Date"])

ema_df = ema_df.set_index("Date").sort_index()

# ========= 5. Merge prices + signals =========
df = price_df.join(sma_df, how="outer").join(ema_df, how="outer")
df = df.sort_index()
df[["signalSMA", "positionSMA", "signalEMA", "positionEMA"]] = (
    df[["signalSMA", "positionSMA", "signalEMA", "positionEMA"]]
    .fillna(0)
    .astype(int)
)

# ========= 6. Combined signal (risk-managed) =========
for c in ["signalSMA", "signalEMA"]:
    df[c] = df[c].clip(-1, 1)

def combine_row(row):
    s_sma = row["signalSMA"]
    s_ema = row["signalEMA"]
    total = s_sma + s_ema

    if total >= 2:
        return 1      # both long
    elif total <= -2:
        return -1     # both short
    elif total == 0:
        return 0      # flat or opposing
    else:
        # one long, the other flat (or one short, other flat)
        return int(np.sign(total))

df["signal_combined"]   = df.apply(combine_row, axis=1)
df["position_combined"] = df["signal_combined"]  # here 1, 0, -1

# ========= 7. PnL on combined signal =========
df["market_ret"]   = df["Close"].pct_change().fillna(0)
df["strategy_ret"] = df["position_combined"].shift(1).fillna(0) * df["market_ret"]
df["strategy_eq"]  = (1 + df["strategy_ret"]).cumprod()

print(df[["Close",
          "signalSMA", "signalEMA",
          "signal_combined", "position_combined"]].tail())

                 Close  signalSMA  signalEMA  signal_combined  \
Date                                                            
2025-12-01  283.100006          0          0                0   
2025-12-02  286.190002          0          0                0   
2025-12-03  284.149994          0          0                0   
2025-12-04  280.700012          0          0                0   
2025-12-05  278.779999          0          0                0   

            position_combined  
Date                           
2025-12-01                  0  
2025-12-02                  0  
2025-12-03                  0  
2025-12-04                  0  
2025-12-05                  0  
